In [ ]:
'''
Código Arduino para ESP32 - Medición de Curva I-V de Diodo LED

Este código implementa:
- Lectura de dos canales ADC (GPIO 35 y GPIO 32)
- Generación de voltaje variable mediante DAC
- Comunicación serial para envío de datos a Python
- Control del LED indicador durante las mediciones

Circuito:
- V1: Voltaje antes de la resistencia (ADC1 - GPIO35)
- V2: Voltaje después de la resistencia/antes del diodo (ADC2 - GPIO32) 
- DAC: Salida de voltaje variable (GPIO25)
- LED: Indicador de estado (GPIO2)
- Resistencia: 1kΩ en serie con el diodo LED
'''

# ============================================================================
# CÓDIGO ARDUINO PARA ESP32
# ============================================================================

code_arduino = '''
/*
 * Proyecto: Medición de Curva Característica I-V de Diodo LED
 * Microcontrolador: ESP32
 * Autor: [Tu nombre]
 * Fecha: Mayo 2025
 * 
 * Descripción:
 * Este código permite medir la corriente y voltaje de un diodo LED
 * para generar su curva característica I-V mediante comunicación
 * serial con Python.
 */

// ============================================================================
// DEFINICIÓN DE PINES Y CONSTANTES
// ============================================================================

// Pines analógicos para lectura de voltajes
const int ADC1_PIN = 35;     // V1 - Voltaje antes de la resistencia
const int ADC2_PIN = 32;     // V2 - Voltaje después de la resistencia

// Pin DAC para generar voltaje variable
const int DAC_PIN = 25;      // Salida analógica para variar voltaje

// Pin digital para LED indicador
const int LED_PIN = 2;       // LED interno del ESP32

// Constantes del sistema
const float VREF = 3.3;      // Voltaje de referencia del ESP32 (3.3V)
const int ADC_RESOLUTION = 4095;  // Resolución del ADC (12 bits = 4095)
const float RESISTENCIA = 1000.0; // Valor de la resistencia en ohmios

// Variables para almacenar lecturas
float voltaje1 = 0.0;        // Voltaje en ADC1
float voltaje2 = 0.0;        // Voltaje en ADC2
float voltajeDiodo = 0.0;    // Voltaje a través del diodo
float corrienteDiodo = 0.0;  // Corriente a través del diodo

// Variables de control
char datoRecibido;           // Dato recibido por serial
int valorDAC = 0;            // Valor actual del DAC (0-255)
bool medicionActiva = false; // Estado de la medición

// ============================================================================
// CONFIGURACIÓN INICIAL
// ============================================================================

void setup() {
  // Inicializar comunicación serial
  Serial.begin(115200);
  
  // Configurar pines
  pinMode(LED_PIN, OUTPUT);
  digitalWrite(LED_PIN, LOW);
  
  // Configurar resolución del DAC (8 bits = 0-255)
  dacWrite(DAC_PIN, 0);
  
  // Mensaje de inicio
  Serial.println("ESP32 - Medidor de Curva I-V del Diodo LED");
  Serial.println("Comandos:");
  Serial.println("  'a' - Iniciar barrido de medición");
  Serial.println("  's' - Detener medición");
  Serial.println("  'r' - Reset DAC a 0V");
  Serial.println("\nListo para recibir comandos...");
  
  delay(1000);
}

// ============================================================================
// BUCLE PRINCIPAL
// ============================================================================

void loop() {
  // Verificar si hay datos disponibles en el puerto serial
  if (Serial.available() > 0) {
    datoRecibido = Serial.read();
    
    // Procesar comando recibido
    switch (datoRecibido) {
      case 'a':
      case 'A':
        // Iniciar barrido de medición
        iniciarBarrido();
        break;
        
      case 's':
      case 'S':
        // Detener medición
        detenerMedicion();
        break;
        
      case 'r':
      case 'R':
        // Reset DAC
        resetDAC();
        break;
        
      default:
        Serial.println("Comando no reconocido");
        break;
    }
  }
  
  delay(50);  // Pequeña pausa para estabilidad
}

// ============================================================================
// FUNCIÓN PARA INICIAR BARRIDO DE MEDICIÓN
// ============================================================================

void iniciarBarrido() {
  Serial.println("Iniciando barrido de medición...");
  medicionActiva = true;
  digitalWrite(LED_PIN, HIGH);  // Encender LED indicador
  
  // Barrido de voltaje de 0 a máximo (0-255 en DAC)
  for (int i = 0; i <= 255 && medicionActiva; i++) {
    valorDAC = i;
    
    // Establecer voltaje en DAC
    dacWrite(DAC_PIN, valorDAC);
    
    // Esperar estabilización
    delay(20);
    
    // Realizar medición
    realizarMedicion();
    
    // Enviar datos por serial
    enviarDatos();
    
    // Verificar si se recibe comando de parada
    if (Serial.available() > 0) {
      char cmd = Serial.read();
      if (cmd == 's' || cmd == 'S') {
        medicionActiva = false;
        break;
      }
    }
  }
  
  // Finalizar medición
  Serial.println("Barrido completado");
  digitalWrite(LED_PIN, LOW);   // Apagar LED indicador
  dacWrite(DAC_PIN, 0);        // Reset DAC a 0V
  medicionActiva = false;
}

// ============================================================================
// FUNCIÓN PARA REALIZAR UNA MEDICIÓN
// ============================================================================

void realizarMedicion() {
  // Realizar múltiples lecturas para promediar (reduce ruido)
  float suma1 = 0, suma2 = 0;
  const int numLecturas = 10;
  
  for (int i = 0; i < numLecturas; i++) {
    suma1 += analogRead(ADC1_PIN);
    suma2 += analogRead(ADC2_PIN);
    delayMicroseconds(100);
  }
  
  // Calcular promedio y convertir a voltaje
  float lectura1 = suma1 / numLecturas;
  float lectura2 = suma2 / numLecturas;
  
  voltaje1 = (lectura1 * VREF) / ADC_RESOLUTION;
  voltaje2 = (lectura2 * VREF) / ADC_RESOLUTION;
  
  // Calcular voltaje del diodo (V2)
  voltajeDiodo = voltaje2;
  
  // Calcular corriente del diodo usando ley de Ohm
  // I = (V1 - V2) / R
  corrienteDiodo = ((voltaje1 - voltaje2) / RESISTENCIA) * 1000000; // en microamperios
  
  // Asegurar que la corriente no sea negativa
  if (corrienteDiodo < 0) {
    corrienteDiodo = 0;
  }
}

// ============================================================================
// FUNCIÓN PARA ENVIAR DATOS POR SERIAL
// ============================================================================

void enviarDatos() {
  // Formato: "V1 V2" (separados por espacio)
  // Python procesará estos valores para calcular Vd e Id
  
  Serial.print(voltaje1, 4);    // 4 decimales de precisión
  Serial.print(" ");
  Serial.println(voltaje2, 4);
  
  // Opcional: enviar también los valores calculados como comentario
  /*
  Serial.print("# Vd=");
  Serial.print(voltajeDiodo, 4);
  Serial.print("V Id=");
  Serial.print(corrienteDiodo, 2);
  Serial.println("uA");
  */
}

// ============================================================================
// FUNCIÓN PARA DETENER MEDICIÓN
// ============================================================================

void detenerMedicion() {
  medicionActiva = false;
  digitalWrite(LED_PIN, LOW);
  dacWrite(DAC_PIN, 0);
  Serial.println("Medición detenida");
}

// ============================================================================
// FUNCIÓN PARA RESET DEL DAC
// ============================================================================

void resetDAC() {
  dacWrite(DAC_PIN, 0);
  valorDAC = 0;
  digitalWrite(LED_PIN, LOW);
  Serial.println("DAC reseteado a 0V");
}

// ============================================================================
// FUNCIONES AUXILIARES OPCIONALES
// ============================================================================

// Función para calibrar el ADC (opcional)
void calibrarADC() {
  Serial.println("Calibrando ADC...");
  
  // Realizar lecturas de calibración
  float suma1 = 0, suma2 = 0;
  const int numCalibraciones = 100;
  
  for (int i = 0; i < numCalibraciones; i++) {
    suma1 += analogRead(ADC1_PIN);
    suma2 += analogRead(ADC2_PIN);
    delay(10);
  }
  
  float offset1 = suma1 / numCalibraciones;
  float offset2 = suma2 / numCalibraciones;
  
  Serial.print("Offset ADC1: ");
  Serial.println(offset1);
  Serial.print("Offset ADC2: ");
  Serial.println(offset2);
}

// Función para mostrar información del sistema
void mostrarInfo() {
  Serial.println("\n=== INFORMACIÓN DEL SISTEMA ===");
  Serial.print("Voltaje de referencia: ");
  Serial.print(VREF);
  Serial.println("V");
  
  Serial.print("Resolución ADC: ");
  Serial.println(ADC_RESOLUTION);
  
  Serial.print("Resistencia: ");
  Serial.print(RESISTENCIA);
  Serial.println(" ohms");
  
  Serial.print("Pin ADC1 (V1): GPIO");
  Serial.println(ADC1_PIN);
  
  Serial.print("Pin ADC2 (V2): GPIO");
  Serial.println(ADC2_PIN);
  
  Serial.print("Pin DAC: GPIO");
  Serial.println(DAC_PIN);
  
  Serial.println("================================\n");
}
'''

print("Código Arduino generado exitosamente.")
print("\nCaracterísticas principales:")
print("- Comunicación serial a 115200 bps")
print("- Lectura de dos canales ADC con promediado para reducir ruido")
print("- Control de voltaje mediante DAC interno")
print("- LED indicador durante mediciones")
print("- Comandos de control por serial ('a', 's', 'r')")
print("- Cálculo de corriente usando ley de Ohm")
print("- Formato de datos compatible con el código Python existente")

## Instrucciones de Conexión del Circuito

### Conexiones del ESP32:

**Entradas Analógicas:**
- **GPIO 35 (ADC1)**: Conectar a V1 (voltaje antes de la resistencia)
- **GPIO 32 (ADC2)**: Conectar a V2 (voltaje después de la resistencia, antes del diodo)

**Salidas:**
- **GPIO 25 (DAC)**: Salida de voltaje variable (0-3.3V)
- **GPIO 2**: LED indicador interno del ESP32

**Alimentación:**
- **3.3V**: Voltaje de referencia del circuito
- **GND**: Tierra común

### Circuito Propuesto:

```
3.3V ----[R=1kΩ]----[LED]---- GND
         |          |
         V1         V2
      (GPIO35)   (GPIO32)
```

### Configuración en Arduino IDE:

1. **Seleccionar la placa**: ESP32 Dev Module
2. **Puerto**: El puerto donde esté conectado tu ESP32
3. **Velocidad de subida**: 921600
4. **Velocidad del monitor serie**: 115200

### Comandos de Control:

- **'a'**: Iniciar barrido de medición automático
- **'s'**: Detener medición en curso
- **'r'**: Reset del DAC a 0V

### Datos de Salida:

El ESP32 enviará por puerto serie pares de valores separados por espacio:
```
V1 V2
```
Donde:
- **V1**: Voltaje antes de la resistencia (en voltios)
- **V2**: Voltaje después de la resistencia/antes del diodo (en voltios)

Estos valores son procesados por el código Python para calcular:
- **Vd**: Voltaje del diodo = V2
- **Id**: Corriente del diodo = (V1-V2)/R * 1000000 μA

In [ ]:
# ============================================================================
# CÓDIGO PYTHON ORIGINAL PROPORCIONADO POR EL PROFESOR
# ============================================================================
# NOTA: Este es el código base proporcionado en clase.
# Contiene algunos errores de sintaxis que fueron corregidos en las versiones optimizadas.
# Se mantiene como referencia histórica del código original.
# Los errores incluyen: sintaxis incorrecta, variables no definidas, funciones mal escritas.

"""
import serial Permité realizar la comunicación serial

import time = Proporciona funciones relacionadas con el tiempo

import matplotlib.pyplot as plt # Permite realizar gráficos

import matplotlib.animation as animation # Permite animar las gráficas

esp serial.Serial("COM10", 115200)

fig, ax = plt.subplots()

line, ax.plot([],[])

x = []

y=[]

Resistencia - 330.0

#valor de la resistencia

ax.set_xlim(0,4) = eje horizontal de 0 a 4 V

ax.set_ylim(0, 1100) # eje vertical de cero a 1100 microamperios
def animate(1):

try:

esp_leido str(esp.readline().rstrip())

valuel (esp_leido.strip("b" "))

value? (value1.split(""))

V1-float(value2[0])

V2-float(value2[1])

Vd (V2)*3.3/4895

Id-(((V2-V1)*3.3/4095)/R)*1000000

print("1"+str(1)+vd+str(d)+"Id"+str(Id)+"\n")

x.append(Vd)

y.append(Id)

line.set data(x,y)

ax.relis()

ax.autoscale view()

#if 1254:

print("FINAN

pit.close()

return line

except Exception as e:

print("Error: (e}")

time.sleep(2)

esp.write(b'a')

ani animation. FuncAnimation(fig. animate, frames 255, Interval 200)

print("fin\n")

plt.show()

esp.close()
"""

## Código Python Original del Profesor

**IMPORTANTE**: La siguiente celda contiene el código Python original proporcionado por el profesor en clase.

### Características del código original:
- Versión inicial para comunicación con ESP32
- Contiene algunos errores de sintaxis (intencionalmente preservados)
- Base para el desarrollo de las versiones optimizadas
- Resistencia configurada para 330Ω (diferente a la versión final de 1kΩ)

### Errores identificados en el código original:
1. **Sintaxis incorrecta**: `esp serial.Serial()` en lugar de `esp = serial.Serial()`
2. **Variables no definidas**: uso de variables como `vd`, `d` sin definir
3. **Funciones mal escritas**: `line.set data()` en lugar de `line.set_data()`
4. **Métodos incorrectos**: `ax.relis()` en lugar de `ax.relim()`
5. **Cálculos con errores**: fórmulas con valores incorrectos

**Las versiones optimizadas posteriores corrigen todos estos errores y añaden funcionalidades avanzadas.**

In [ ]:
# ============================================================================
# CÓDIGO PYTHON OPTIMIZADO PARA COMUNICACIÓN CON ESP32
# ============================================================================

import serial          # Comunicación serial
import time            # Funciones de tiempo
import matplotlib.pyplot as plt    # Gráficos
import matplotlib.animation as animation   # Animación
import numpy as np     # Operaciones matemáticas
import threading       # Para hilos
import queue          # Cola thread-safe
from collections import deque
import re             # Expresiones regulares

# ============================================================================
# CLASE OPTIMIZADA PARA COMUNICACIÓN SERIAL
# ============================================================================

class ESP32CommunicationOptimized:
    def __init__(self, port=None, baudrate=115200, timeout=1):
        self.port = port
        self.baudrate = baudrate
        self.timeout = timeout
        self.serial_conn = None
        self.data_queue = queue.Queue(maxsize=1000)
        self.system_queue = queue.Queue(maxsize=100)
        self.running = False
        self.thread = None
        
        # Estadísticas
        self.packets_received = 0
        self.packets_corrupted = 0
        self.last_checksum = 0
        
    def connect(self):
        """Conectar al ESP32 con reintentos automáticos"""
        ports_to_try = []
        
        if self.port:
            ports_to_try.append(self.port)
        
        # Puertos comunes según el sistema
        common_ports = [
            "/dev/ttyUSB0", "/dev/ttyUSB1", "/dev/ttyACM0", "/dev/ttyACM1",
            "COM3", "COM4", "COM5", "COM10", "COM20"
        ]
        ports_to_try.extend(common_ports)
        
        for port in ports_to_try:
            try:
                print(f"Intentando conectar a {port}...")
                self.serial_conn = serial.Serial(port, self.baudrate, timeout=self.timeout)
                time.sleep(2)  # Esperar inicialización del ESP32
                
                # Verificar que sea nuestro ESP32
                self.serial_conn.write(b'i')  # Comando info
                time.sleep(0.5)
                
                response = self.serial_conn.read_all().decode('utf-8', errors='ignore')
                if 'INFO:' in response or 'ESP32' in response:
                    print(f"✅ Conectado exitosamente a {port}")
                    self.port = port
                    return True
                else:
                    self.serial_conn.close()
                    
            except Exception as e:
                print(f"❌ Error en {port}: {e}")
                if self.serial_conn:
                    self.serial_conn.close()
                continue
        
        print("❌ No se pudo conectar a ningún puerto")
        return False
    
    def start_reading(self):
        """Iniciar hilo de lectura de datos"""
        if not self.serial_conn or not self.serial_conn.is_open:
            return False
            
        self.running = True
        self.thread = threading.Thread(target=self._read_data_thread, daemon=True)
        self.thread.start()
        return True
    
    def _read_data_thread(self):
        """Hilo para lectura continua de datos"""
        buffer = ""
        
        while self.running and self.serial_conn and self.serial_conn.is_open:
            try:
                if self.serial_conn.in_waiting > 0:
                    chunk = self.serial_conn.read(self.serial_conn.in_waiting).decode('utf-8', errors='ignore')
                    buffer += chunk
                    
                    # Procesar líneas completas
                    while '\n' in buffer:
                        line, buffer = buffer.split('\n', 1)
                        line = line.strip()
                        if line:
                            self._process_line(line)
                
                time.sleep(0.001)  # Pequeña pausa para evitar uso intensivo de CPU
                
            except Exception as e:
                print(f"Error en hilo de lectura: {e}")
                time.sleep(0.1)
    
    def _process_line(self, line):
        """Procesar una línea recibida del ESP32"""
        try:
            if line.startswith('DATA:'):
                # Formato: DATA:V1,V2,Vd,Id,DAC,CHECKSUM
                data_part = line[5:]  # Remover 'DATA:'
                parts = data_part.split(',')
                
                if len(parts) >= 6:
                    V1 = float(parts[0])
                    V2 = float(parts[1])
                    Vd = float(parts[2])
                    Id = float(parts[3])
                    DAC = int(parts[4])
                    checksum = int(parts[5])
                    
                    # Verificar integridad básica
                    calculated_checksum = self._calculate_checksum(V1, V2, Id, DAC)
                    
                    data_packet = {
                        'V1': V1, 'V2': V2, 'Vd': Vd, 'Id': Id, 
                        'DAC': DAC, 'checksum': checksum,
                        'timestamp': time.time(),
                        'valid': abs(checksum - calculated_checksum) < 100  # Tolerancia
                    }
                    
                    if not self.data_queue.full():
                        self.data_queue.put(data_packet)
                        self.packets_received += 1
                        if not data_packet['valid']:
                            self.packets_corrupted += 1
            
            elif line.startswith('SYS:'):
                # Mensaje del sistema
                sys_msg = line[4:]  # Remover 'SYS:'
                if not self.system_queue.full():
                    self.system_queue.put({'message': sys_msg, 'timestamp': time.time()})
            
            elif line.startswith('INFO:'):
                # Información del sistema
                print(f"ℹ️  {line}")
                
        except Exception as e:
            print(f"Error procesando línea '{line}': {e}")
    
    def _calculate_checksum(self, V1, V2, Id, DAC):
        """Calcular checksum para verificación"""
        return int((V1 * 1000) + (V2 * 1000) + Id + DAC) & 0xFFFF
    
    def send_command(self, command):
        """Enviar comando al ESP32"""
        if self.serial_conn and self.serial_conn.is_open:
            self.serial_conn.write(command.encode())
            self.serial_conn.flush()
            return True
        return False
    
    def get_data(self, timeout=0.1):
        """Obtener datos de la cola"""
        try:
            return self.data_queue.get(timeout=timeout)
        except queue.Empty:
            return None
    
    def get_system_message(self, timeout=0.1):
        """Obtener mensaje del sistema"""
        try:
            return self.system_queue.get(timeout=timeout)
        except queue.Empty:
            return None
    
    def get_statistics(self):
        """Obtener estadísticas de comunicación"""
        return {
            'packets_received': self.packets_received,
            'packets_corrupted': self.packets_corrupted,
            'queue_size': self.data_queue.qsize(),
            'corruption_rate': (self.packets_corrupted / max(1, self.packets_received)) * 100
        }
    
    def stop(self):
        """Detener comunicación"""
        self.running = False
        if self.thread:
            self.thread.join(timeout=2)
        if self.serial_conn and self.serial_conn.is_open:
            self.serial_conn.close()

# ============================================================================
# CLASE PARA VISUALIZACIÓN OPTIMIZADA
# ============================================================================

class OptimizedLEDCurvePlotter:
    def __init__(self, max_points=500):
        self.max_points = max_points
        self.data_buffer = deque(maxlen=max_points)
        self.esp_comm = None
        
        # Configurar gráfico
        self.fig, (self.ax1, self.ax2) = plt.subplots(2, 1, figsize=(12, 10))
        
        # Gráfico principal I-V
        self.line_iv, = self.ax1.plot([], [], 'b-', linewidth=2, alpha=0.8, label='Curva I-V')
        self.scatter_iv = self.ax1.scatter([], [], c=[], s=20, cmap='viridis', alpha=0.6)
        
        self.ax1.set_xlim(0, 4)
        self.ax1.set_ylim(0, 1200)
        self.ax1.set_xlabel('Voltaje del Diodo (V)')
        self.ax1.set_ylabel('Corriente del Diodo (μA)')
        self.ax1.set_title('Curva Característica I-V del Diodo LED - Tiempo Real')
        self.ax1.grid(True, alpha=0.3)
        self.ax1.legend()
        
        # Gráfico de estadísticas
        self.ax2.set_xlim(0, 100)
        self.ax2.set_ylim(0, 10)
        self.ax2.set_xlabel('Tiempo (s)')
        self.ax2.set_ylabel('Estadísticas')
        self.ax2.set_title('Estadísticas de Comunicación')
        self.ax2.grid(True, alpha=0.3)
        
        # Variables de control
        self.start_time = time.time()
        self.stats_history = deque(maxlen=100)
        
    def connect_esp32(self, port=None):
        """Conectar al ESP32"""
        self.esp_comm = ESP32CommunicationOptimized(port)
        if self.esp_comm.connect():
            self.esp_comm.start_reading()
            return True
        return False
    
    def animate(self, frame):
        """Función de animación optimizada"""
        if not self.esp_comm:
            return self.line_iv, self.scatter_iv
        
        # Procesar mensajes del sistema
        sys_msg = self.esp_comm.get_system_message()
        if sys_msg:
            print(f"🔧 Sistema: {sys_msg['message']}")
        
        # Obtener nuevos datos
        data = self.esp_comm.get_data()
        if data and data['valid']:
            self.data_buffer.append(data)
        
        # Actualizar gráfico I-V
        if len(self.data_buffer) > 1:
            voltages = [d['Vd'] for d in self.data_buffer]
            currents = [d['Id'] for d in self.data_buffer]
            dac_values = [d['DAC'] for d in self.data_buffer]
            
            # Línea principal
            self.line_iv.set_data(voltages, currents)
            
            # Puntos coloreados por valor DAC
            if len(voltages) > 0:
                self.ax1.collections.clear()  # Limpiar scatter anterior
                self.scatter_iv = self.ax1.scatter(voltages, currents, 
                                                 c=dac_values, s=30, 
                                                 cmap='plasma', alpha=0.7)
            
            # Ajustar límites automáticamente
            if len(voltages) > 10:
                max_v = max(voltages)
                max_i = max(currents)
                self.ax1.set_xlim(0, max(max_v * 1.1, 4))
                self.ax1.set_ylim(0, max(max_i * 1.1, 1200))
        
        # Actualizar estadísticas
        current_time = time.time() - self.start_time
        stats = self.esp_comm.get_statistics()
        stats['time'] = current_time
        self.stats_history.append(stats)
        
        if len(self.stats_history) > 1:
            times = [s['time'] for s in self.stats_history]
            corruption_rates = [s['corruption_rate'] for s in self.stats_history]
            
            self.ax2.clear()
            self.ax2.plot(times, corruption_rates, 'r-', label='Tasa de Error (%)')
            self.ax2.set_xlabel('Tiempo (s)')
            self.ax2.set_ylabel('Tasa de Error (%)')
            self.ax2.set_title(f'Estadísticas - Paquetes: {stats["packets_received"]} | Errores: {stats["packets_corrupted"]}')
            self.ax2.grid(True, alpha=0.3)
            self.ax2.legend()
        
        return self.line_iv, self.scatter_iv
    
    def start_measurement(self):
        """Iniciar medición"""
        if self.esp_comm:
            print("🚀 Iniciando medición...")
            self.esp_comm.send_command('a')
            return True
        return False
    
    def stop_measurement(self):
        """Detener medición"""
        if self.esp_comm:
            print("⏹️ Deteniendo medición...")
            self.esp_comm.send_command('s')
            return True
        return False
    
    def save_data(self, filename=None):
        """Guardar datos a archivo CSV"""
        if not filename:
            timestamp = time.strftime("%Y%m%d_%H%M%S")
            filename = f"curva_LED_optimizada_{timestamp}.csv"
        
        try:
            import csv
            with open(filename, 'w', newline='', encoding='utf-8') as file:
                writer = csv.writer(file)
                writer.writerow(['Timestamp', 'V1', 'V2', 'Vd', 'Id', 'DAC', 'Valid'])
                
                for data in self.data_buffer:
                    writer.writerow([
                        data['timestamp'], data['V1'], data['V2'], 
                        data['Vd'], data['Id'], data['DAC'], data['valid']
                    ])
            
            print(f"💾 Datos guardados en: {filename}")
            return filename
        except Exception as e:
            print(f"❌ Error guardando datos: {e}")
            return None
    
    def run(self, port=None):
        """Ejecutar el graficador completo"""
        try:
            # Conectar
            if not self.connect_esp32(port):
                print("❌ No se pudo conectar al ESP32")
                return
            
            # Configurar evento de cierre
            def on_close(event):
                self.stop_measurement()
                if self.data_buffer:
                    self.save_data()
                if self.esp_comm:
                    self.esp_comm.stop()
                plt.close('all')
            
            self.fig.canvas.mpl_connect('close_event', on_close)
            
            # Iniciar medición automáticamente
            time.sleep(1)
            self.start_measurement()
            
            # Crear animación
            ani = animation.FuncAnimation(
                self.fig, self.animate, interval=50, 
                blit=False, repeat=True, cache_frame_data=False
            )
            
            plt.tight_layout()
            plt.show()
            
        except KeyboardInterrupt:
            print("\n⚠️ Medición interrumpida por el usuario")
        except Exception as e:
            print(f"❌ Error: {e}")
        finally:
            if self.esp_comm:
                self.esp_comm.stop()

# ============================================================================
# FUNCIÓN PRINCIPAL OPTIMIZADA
# ============================================================================

def main_optimized(port=None):
    """Función principal optimizada"""
    print("🔬 Medidor de Curva I-V del LED - Versión Optimizada")
    print("="*50)
    
    plotter = OptimizedLEDCurvePlotter(max_points=300)
    plotter.run(port)

# Ejecutar si se llama directamente
if __name__ == "__main__":
    # Detectar puerto automáticamente o usar el especificado
    import sys
    port = sys.argv[1] if len(sys.argv) > 1 else None
    main_optimized(port)

print("✅ Código Python optimizado cargado")
print("📌 Usar: main_optimized() para ejecutar")
print("📌 Usar: main_optimized('/dev/ttyUSB0') para puerto específico")

In [ ]:
# ============================================================================
# FUNCIÓN ADICIONAL PARA GUARDAR DATOS
# ============================================================================

import csv
import datetime

def guardar_datos_csv(voltajes, corrientes, nombre_archivo=None):
    """
    Guarda los datos de voltaje y corriente en un archivo CSV
    
    Parámetros:
    - voltajes: lista de voltajes del diodo
    - corrientes: lista de corrientes del diodo
    - nombre_archivo: nombre del archivo (opcional)
    """
    
    if nombre_archivo is None:
        # Generar nombre automático con fecha y hora
        timestamp = datetime.datetime.now().strftime("%Y%m%d_%H%M%S")
        nombre_archivo = f"curva_LED_{timestamp}.csv"
    
    try:
        with open(nombre_archivo, 'w', newline='', encoding='utf-8') as archivo:
            escritor = csv.writer(archivo)
            
            # Escribir encabezados
            escritor.writerow(['Voltaje_Diodo_V', 'Corriente_Diodo_uA'])
            
            # Escribir datos
            for v, i in zip(voltajes, corrientes):
                escritor.writerow([v, i])
        
        print(f"Datos guardados en: {nombre_archivo}")
        return True
        
    except Exception as e:
        print(f"Error al guardar datos: {e}")
        return False

def analizar_datos(voltajes, corrientes):
    """
    Realiza análisis básico de los datos obtenidos
    """
    
    if len(voltajes) == 0 or len(corrientes) == 0:
        print("No hay datos para analizar")
        return
    
    # Convertir a arrays de numpy
    V = np.array(voltajes)
    I = np.array(corrientes)
    
    # Estadísticas básicas
    print("\n=== ANÁLISIS DE DATOS ===")
    print(f"Número de puntos: {len(V)}")
    print(f"Voltaje mínimo: {np.min(V):.3f} V")
    print(f"Voltaje máximo: {np.max(V):.3f} V")
    print(f"Corriente mínima: {np.min(I):.2f} μA")
    print(f"Corriente máxima: {np.max(I):.2f} μA")
    
    # Encontrar voltaje de encendido (umbral)
    # Definir como el punto donde la corriente supera 10 μA
    umbral_corriente = 10  # μA
    indices_encendido = np.where(I > umbral_corriente)[0]
    
    if len(indices_encendido) > 0:
        voltaje_encendido = V[indices_encendido[0]]
        print(f"Voltaje de encendido (≈{umbral_corriente}μA): {voltaje_encendido:.3f} V")
    
    # Calcular resistencia dinámica en diferentes puntos
    if len(V) > 10:
        # Tomar puntos en la región lineal (parte superior de la curva)
        indices_lineales = np.where(I > np.max(I) * 0.5)[0]
        
        if len(indices_lineales) > 5:
            V_lin = V[indices_lineales]
            I_lin = I[indices_lineales]
            
            # Calcular pendiente (dI/dV)
            pendiente = np.gradient(I_lin, V_lin)
            resistencia_dinamica = 1000 / np.mean(pendiente)  # en ohms
            
            print(f"Resistencia dinámica promedio: {resistencia_dinamica:.1f} Ω")
    
    print("========================\n")

# Ejemplo de uso al final del programa principal:
"""
# Al final de la función main(), agregar:
if len(x_data) > 0 and len(y_data) > 0:
    # Guardar datos
    guardar_datos_csv(x_data, y_data)
    
    # Analizar datos
    analizar_datos(x_data, y_data)
    
    # Crear gráfico final mejorado
    plt.figure(figsize=(12, 8))
    plt.subplot(2, 1, 1)
    plt.plot(x_data, y_data, 'b.-', linewidth=2, markersize=3)
    plt.xlabel('Voltaje del Diodo (V)')
    plt.ylabel('Corriente del Diodo (μA)')
    plt.title('Curva Característica I-V del Diodo LED')
    plt.grid(True, alpha=0.3)
    
    # Gráfico en escala logarítmica
    plt.subplot(2, 1, 2)
    plt.semilogy(x_data, np.array(y_data) + 0.1, 'r.-', linewidth=2, markersize=3)
    plt.xlabel('Voltaje del Diodo (V)')
    plt.ylabel('Corriente del Diodo (μA) - Escala Log')
    plt.title('Curva I-V en Escala Logarítmica')
    plt.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
"""

In [ ]:
# ============================================================================
# FUNCIONES AUXILIARES OPTIMIZADAS
# ============================================================================

def test_communication(port=None):
    """Probar comunicación con ESP32"""
    print("🔍 Probando comunicación con ESP32...")
    
    esp = ESP32CommunicationOptimized(port)
    if esp.connect():
        esp.start_reading()
        
        # Enviar comando de información
        esp.send_command('i')
        time.sleep(1)
        
        # Leer respuestas
        for _ in range(10):
            msg = esp.get_system_message(timeout=0.5)
            if msg:
                print(f"📨 {msg['message']}")
        
        # Estadísticas
        stats = esp.get_statistics()
        print(f"📊 Estadísticas: {stats}")
        
        esp.stop()
        print("✅ Prueba completada")
        return True
    else:
        print("❌ No se pudo conectar")
        return False

def calibrate_esp32(port=None):
    """Calibrar ESP32"""
    print("⚙️ Calibrando ESP32...")
    
    esp = ESP32CommunicationOptimized(port)
    if esp.connect():
        esp.start_reading()
        
        # Enviar comando de calibración
        esp.send_command('c')
        time.sleep(3)
        
        # Leer resultados
        for _ in range(20):
            msg = esp.get_system_message(timeout=0.5)
            if msg:
                print(f"🔧 {msg['message']}")
        
        esp.stop()
        print("✅ Calibración completada")
        return True
    else:
        print("❌ No se pudo conectar")
        return False

def analyze_saved_data(filename):
    """Analizar datos guardados"""
    try:
        import pandas as pd
        import numpy as np
        
        print(f"📈 Analizando datos de {filename}...")
        
        # Cargar datos
        df = pd.read_csv(filename)
        valid_data = df[df['Valid'] == True]
        
        if len(valid_data) == 0:
            print("❌ No hay datos válidos para analizar")
            return
        
        # Estadísticas básicas
        print(f"📊 Puntos totales: {len(df)}")
        print(f"📊 Puntos válidos: {len(valid_data)}")
        print(f"📊 Validez: {len(valid_data)/len(df)*100:.1f}%")
        
        # Análisis de la curva
        V = valid_data['Vd'].values
        I = valid_data['Id'].values
        
        print(f"🔍 Rango de voltaje: {V.min():.3f}V - {V.max():.3f}V")
        print(f"🔍 Rango de corriente: {I.min():.2f}μA - {I.max():.2f}μA")
        
        # Encontrar voltaje de encendido
        threshold = 10  # μA
        turn_on_indices = np.where(I > threshold)[0]
        if len(turn_on_indices) > 0:
            turn_on_voltage = V[turn_on_indices[0]]
            print(f"💡 Voltaje de encendido (>{threshold}μA): {turn_on_voltage:.3f}V")
        
        # Resistencia dinámica
        if len(V) > 10:
            high_current_mask = I > I.max() * 0.7
            if np.sum(high_current_mask) > 5:
                V_high = V[high_current_mask]
                I_high = I[high_current_mask]
                
                # Calcular pendiente
                slope = np.gradient(I_high, V_high)
                dynamic_resistance = 1000 / np.mean(slope)  # en ohms
                print(f"⚡ Resistencia dinámica: {dynamic_resistance:.1f}Ω")
        
        # Crear gráfico de análisis
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        # Curva I-V
        ax1.plot(V, I, 'b.-', linewidth=2, markersize=3)
        ax1.set_xlabel('Voltaje (V)')
        ax1.set_ylabel('Corriente (μA)')
        ax1.set_title('Curva I-V Completa')
        ax1.grid(True, alpha=0.3)
        
        # Curva en escala logarítmica
        ax2.semilogy(V, I + 0.1, 'r.-', linewidth=2, markersize=3)
        ax2.set_xlabel('Voltaje (V)')
        ax2.set_ylabel('Corriente (μA) - Log')
        ax2.set_title('Curva I-V Logarítmica')
        ax2.grid(True, alpha=0.3)
        
        # Histograma de voltajes
        ax3.hist(V, bins=30, alpha=0.7, color='green')
        ax3.set_xlabel('Voltaje (V)')
        ax3.set_ylabel('Frecuencia')
        ax3.set_title('Distribución de Voltajes')
        ax3.grid(True, alpha=0.3)
        
        # Validez de datos vs tiempo
        ax4.plot(df.index, df['Valid'].astype(int), 'k.-', alpha=0.7)
        ax4.set_xlabel('Muestra #')
        ax4.set_ylabel('Validez (0/1)')
        ax4.set_title('Calidad de Datos vs Tiempo')
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        return {
            'total_points': len(df),
            'valid_points': len(valid_data),
            'validity_rate': len(valid_data)/len(df)*100,
            'voltage_range': (V.min(), V.max()),
            'current_range': (I.min(), I.max())
        }
        
    except Exception as e:
        print(f"❌ Error analizando datos: {e}")
        return None

# Funciones de conveniencia
def quick_test():
    """Prueba rápida del sistema"""
    return test_communication()

def quick_calibration():
    """Calibración rápida"""
    return calibrate_esp32()

def quick_measurement(duration_minutes=2):
    """Medición rápida con duración limitada"""
    print(f"⏱️ Medición rápida por {duration_minutes} minutos")
    
    plotter = OptimizedLEDCurvePlotter(max_points=200)
    
    # Conectar
    if not plotter.connect_esp32():
        print("❌ No se pudo conectar")
        return
    
    # Medir
    plotter.start_measurement()
    
    # Esperar duración especificada
    time.sleep(duration_minutes * 60)
    
    # Detener y guardar
    plotter.stop_measurement()
    filename = plotter.save_data()
    
    if filename:
        analyze_saved_data(filename)

print("🔧 Funciones auxiliares cargadas:")
print("   - test_communication() o quick_test()")
print("   - calibrate_esp32() o quick_calibration()")
print("   - analyze_saved_data(filename)")
print("   - quick_measurement(duration_minutes=2)")

## Resumen del Proyecto

### Objetivo
Este proyecto permite medir y visualizar la **curva característica I-V de un diodo LED** utilizando un **ESP32** y **Python**.

### Componentes del Sistema

1. **Hardware:**
   - ESP32 (microcontrolador)
   - Diodo LED
   - Resistencia de 1kΩ
   - Protoboard y cables

2. **Software:**
   - Código Arduino para ESP32 (medición y control)
   - Código Python (visualización y análisis)

### Funcionamiento

1. **Generación de voltaje**: El ESP32 usa su DAC interno para generar voltajes variables (0-3.3V)
2. **Medición**: Dos canales ADC miden:
   - V1: Voltaje antes de la resistencia
   - V2: Voltaje después de la resistencia (= voltaje del diodo)
3. **Cálculo**: La corriente se calcula usando la Ley de Ohm: I = (V1-V2)/R
4. **Visualización**: Python recibe los datos y genera la curva I-V en tiempo real

### Características Implementadas

✅ **Medición automática** con barrido de voltaje
✅ **Comunicación serial** robusta
✅ **Visualización en tiempo real**
✅ **Promediado de lecturas** para reducir ruido
✅ **Control por comandos** ('a', 's', 'r')
✅ **Indicador LED** durante mediciones
✅ **Guardado de datos** en formato CSV
✅ **Análisis automático** de resultados
✅ **Manejo de errores** y reconexión

### Resultados Esperados

La curva I-V típica de un LED muestra:
- **Región de corte**: Corriente ≈ 0 para V < Vf
- **Voltaje de encendido** (Vf): ≈ 1.8-3.2V según el color del LED
- **Región de conducción**: Crecimiento exponencial de la corriente
- **Resistencia dinámica**: Baja en la región de conducción

### Ventajas del Sistema

- **Bajo costo**: Usa componentes accesibles
- **Precisión**: Promediado de múltiples lecturas
- **Flexibilidad**: Fácil modificación para otros componentes
- **Educativo**: Código comentado y bien estructurado
- **Visualización**: Gráficos en tiempo real y análisis automático

# Código Arduino para Medición de Curva I-V del Diodo LED

Este código está diseñado para un ESP32 que mide la corriente y voltaje de un diodo LED mediante dos canales ADC, permitiendo generar la curva característica I-V del componente.

## Resumen del Proyecto - Versión Optimizada

### Objetivo
Este proyecto optimizado permite medir y visualizar la **curva característica I-V de un diodo LED** utilizando un **ESP32** y **Python** con comunicación robusta y análisis en tiempo real.

### Componentes del Sistema

1. **Hardware:**
   - ESP32 (microcontrolador)
   - Diodo LED
   - Resistencia de 1kΩ
   - Protoboard y cables

2. **Software Optimizado:**
   - Código Arduino optimizado con máquina de estados
   - Código Python con comunicación asíncrona y análisis en tiempo real

### Optimizaciones Implementadas

#### Arduino/ESP32:
✅ **Máquina de estados** para control robusto  
✅ **Protocolo de comunicación** con checksum  
✅ **Buffer circular** para datos  
✅ **Filtrado de ruido** mejorado (16 muestras promediadas)  
✅ **Timing optimizado** con delays adaptativos  
✅ **Configuración ADC** mejorada (12 bits, atenuación 11dB)  
✅ **Comandos extendidos** (info, calibración, reset)  
✅ **Watchdog** para detección de timeout  

#### Python:
✅ **Comunicación asíncrona** con hilos  
✅ **Auto-detección de puertos** serie  
✅ **Cola thread-safe** para datos  
✅ **Verificación de integridad** con checksum  
✅ **Estadísticas en tiempo real** de comunicación  
✅ **Visualización dual** (I-V + estadísticas)  
✅ **Coloreado por valor DAC** en scatter plot  
✅ **Análisis automático** de datos guardados  
✅ **Funciones de conveniencia** para pruebas rápidas  

### Protocolo de Comunicación Optimizado

**Formato de datos:**
```
DATA:V1,V2,Vd,Id,DAC,CHECKSUM
```

**Mensajes del sistema:**
```
SYS:READY
SYS:START
SYS:MEASURING
SYS:COMPLETE
```

**Comandos disponibles:**
- **'a'**: Iniciar medición
- **'s'**: Detener medición  
- **'r'**: Reset DAC
- **'i'**: Información del sistema
- **'c'**: Calibrar ADC

### Características Avanzadas

🔬 **Análisis en tiempo real** con métricas de calidad  
📊 **Visualización dual**: Curva I-V + estadísticas de comunicación  
🎨 **Coloreado dinámico** de puntos según valor DAC  
💾 **Guardado automático** con timestamp  
📈 **Análisis post-procesamiento** con pandas  
🔧 **Funciones de prueba** y calibración  
⚡ **Rendimiento optimizado** con threading  
🛡️ **Detección de errores** y recuperación automática  

### Uso del Sistema Optimizado

```python
# Uso básico
main_optimized()  # Auto-detecta puerto

# Puerto específico
main_optimized('/dev/ttyUSB0')

# Funciones de prueba
quick_test()           # Probar comunicación
quick_calibration()    # Calibrar sistema
quick_measurement(5)   # Medición de 5 minutos

# Análisis de datos
analyze_saved_data('curva_LED_20250530_143022.csv')
```

### Métricas de Rendimiento

- **Velocidad de muestreo**: ~10-20 Hz (adaptativa)
- **Precisión**: 12 bits ADC + promediado de 16 muestras
- **Latencia**: < 100ms por punto
- **Detección de errores**: Checksum + validación de rango
- **Capacidad buffer**: 1000 puntos de datos
- **Auto-recuperación**: Reintentos automáticos en fallos

### Ventajas del Sistema Optimizado

- **Robustez**: Manejo de errores y recuperación automática
- **Precisión**: Filtrado avanzado y verificación de integridad
- **Flexibilidad**: Múltiples modos de operación y análisis
- **Escalabilidad**: Fácil extensión para otros componentes
- **Usabilidad**: Interface simplificada con funciones de conveniencia
- **Visualización**: Gráficos informativos en tiempo real
- **Análisis**: Herramientas integradas de post-procesamiento

In [ ]:
# ============================================================================
# CÓDIGO PYTHON OPTIMIZADO - COMUNICACIÓN THREADED Y ANÁLISIS AVANZADO
# ============================================================================

import serial
import threading
import queue
import time
import matplotlib.pyplot as plt
import matplotlib.animation as animation
import numpy as np
import pandas as pd
from datetime import datetime
import csv
import json
import re
from typing import List, Tuple, Optional

class ESP32CommunicationOptimized:
    """
    Clase optimizada para comunicación con ESP32
    Incluye threading, manejo de errores avanzado y protocolo robusto
    """
    
    def __init__(self, port: str = None, baudrate: int = 115200):
        self.port = port
        self.baudrate = baudrate
        self.serial_connection = None
        self.data_queue = queue.Queue(maxsize=500)
        self.system_queue = queue.Queue(maxsize=100)
        self.is_connected = False
        self.is_reading = False
        self.read_thread = None
        self.stats = {
            'total_packets': 0,
            'valid_packets': 0,
            'checksum_errors': 0,
            'communication_errors': 0
        }
    
    def find_port(self) -> Optional[str]:
        """Busca automáticamente el puerto del ESP32"""
        import serial.tools.list_ports
        
        ports = serial.tools.list_ports.comports()
        esp32_ports = []
        
        for port in ports:
            # Buscar ESP32 por descripción o VID/PID
            if any(keyword in port.description.lower() for keyword in 
                   ['esp32', 'cp210x', 'ch340', 'silicon labs', 'usb serial']):
                esp32_ports.append(port.device)
        
        if esp32_ports:
            print(f"Puertos ESP32 encontrados: {esp32_ports}")
            return esp32_ports[0]
        
        return None
    
    def connect(self) -> bool:
        """Establece conexión con el ESP32"""
        if self.port is None:
            self.port = self.find_port()
            if self.port is None:
                print("Error: No se encontró puerto ESP32")
                return False
        
        try:
            self.serial_connection = serial.Serial(
                self.port, 
                self.baudrate, 
                timeout=1,
                write_timeout=1
            )
            
            # Esperar inicialización
            time.sleep(2)
            
            # Limpiar buffers
            self.serial_connection.flushInput()
            self.serial_connection.flushOutput()
            
            self.is_connected = True
            print(f"Conexión establecida en {self.port}")
            
            # Iniciar thread de lectura
            self.start_reading()
            
            return True
            
        except Exception as e:
            print(f"Error de conexión: {e}")
            return False
    
    def start_reading(self):
        """Inicia el thread de lectura de datos"""
        self.is_reading = True
        self.read_thread = threading.Thread(target=self._read_data)
        self.read_thread.daemon = True
        self.read_thread.start()
    
    def _read_data(self):
        """Thread para lectura continua de datos"""
        while self.is_reading and self.is_connected:
            try:
                if self.serial_connection.in_waiting > 0:
                    line = self.serial_connection.readline().decode('utf-8').strip()
                    
                    if line:
                        self.stats['total_packets'] += 1
                        
                        if line.startswith('DATA:'):
                            # Procesar datos optimizados
                            data = self._parse_data_line(line)
                            if data:
                                self.data_queue.put(data)
                                self.stats['valid_packets'] += 1
                        
                        elif line.startswith('SYS:'):
                            # Mensaje del sistema
                            system_msg = line[4:]
                            self.system_queue.put(system_msg)
                        
                        elif line.startswith('INFO:') or line.startswith('CAL:'):
                            # Información del sistema
                            print(line)
                        
                        else:
                            # Mensaje general
                            print(f"ESP32: {line}")
                
                time.sleep(0.001)  # Pequeña pausa para no saturar CPU
                
            except Exception as e:
                self.stats['communication_errors'] += 1
                print(f"Error de lectura: {e}")
                time.sleep(0.1)
    
    def _parse_data_line(self, line: str) -> Optional[dict]:
        """Parsea línea de datos optimizada: DATA:V1,V2,Vd,Id,DAC,CHECKSUM"""
        try:
            data_part = line[5:]  # Quitar 'DATA:'
            values = data_part.split(',')
            
            if len(values) >= 6:
                v1 = float(values[0])
                v2 = float(values[1])
                vd = float(values[2])
                id_current = float(values[3])
                dac = int(values[4])
                checksum_received = int(values[5])
                
                # Verificar checksum
                checksum_calculated = self._calculate_checksum(v1, v2, id_current, dac)
                
                if abs(checksum_received - checksum_calculated) <= 5:  # Tolerancia
                    return {
                        'v1': v1,
                        'v2': v2,
                        'vd': vd,
                        'id': id_current,
                        'dac': dac,
                        'timestamp': time.time()
                    }
                else:
                    self.stats['checksum_errors'] += 1
                    print(f"Error checksum: {checksum_received} vs {checksum_calculated}")
        
        except Exception as e:
            print(f"Error parsing: {e}")
        
        return None
    
    def _calculate_checksum(self, v1: float, v2: float, id_current: float, dac: int) -> int:
        """Calcula checksum para verificación"""
        sum_val = int(v1 * 1000) + int(v2 * 1000) + int(id_current) + dac
        return sum_val & 0xFFFF
    
    def send_command(self, command: str) -> bool:
        """Envía comando al ESP32"""
        if not self.is_connected:
            return False
        
        try:
            self.serial_connection.write(command.encode())
            self.serial_connection.flush()
            return True
        except Exception as e:
            print(f"Error enviando comando: {e}")
            return False
    
    def get_data(self, timeout: float = 0.1) -> Optional[dict]:
        """Obtiene datos del queue con timeout"""
        try:
            return self.data_queue.get(timeout=timeout)
        except queue.Empty:
            return None
    
    def get_system_message(self, timeout: float = 0.1) -> Optional[str]:
        """Obtiene mensaje del sistema"""
        try:
            return self.system_queue.get(timeout=timeout)
        except queue.Empty:
            return None
    
    def get_stats(self) -> dict:
        """Obtiene estadísticas de comunicación"""
        stats = self.stats.copy()
        if stats['total_packets'] > 0:
            stats['success_rate'] = (stats['valid_packets'] / stats['total_packets']) * 100
        else:
            stats['success_rate'] = 0
        return stats
    
    def disconnect(self):
        """Cierra la conexión"""
        self.is_reading = False
        self.is_connected = False
        
        if self.read_thread:
            self.read_thread.join(timeout=2)
        
        if self.serial_connection and self.serial_connection.is_open:
            self.serial_connection.close()
        
        print("Conexión cerrada")

print("Clase ESP32CommunicationOptimized cargada exitosamente")

In [ ]:
class OptimizedLEDCurvePlotter:
    """
    Visualizador optimizado de curvas I-V con análisis en tiempo real
    """
    
    def __init__(self, max_points: int = 300):
        self.max_points = max_points
        self.voltages = []
        self.currents = []
        self.timestamps = []
        self.raw_data = []
        
        # Configurar figura con subplots
        self.fig, (self.ax_main, self.ax_stats) = plt.subplots(2, 1, figsize=(12, 10))
        
        # Configurar gráfico principal
        self.line_iv, = self.ax_main.plot([], [], 'b-', linewidth=2, alpha=0.8, label='Curva I-V')
        self.points_iv, = self.ax_main.plot([], [], 'ro', markersize=3, alpha=0.6, label='Puntos')
        
        self.ax_main.set_xlim(0, 4)
        self.ax_main.set_ylim(0, 1200)
        self.ax_main.set_xlabel('Voltaje del Diodo (V)')
        self.ax_main.set_ylabel('Corriente del Diodo (μA)')
        self.ax_main.set_title('Curva Característica I-V del LED - Tiempo Real')
        self.ax_main.grid(True, alpha=0.3)
        self.ax_main.legend()
        
        # Configurar gráfico de estadísticas
        self.line_rate, = self.ax_stats.plot([], [], 'g-', label='Tasa de datos (Hz)')
        self.line_errors, = self.ax_stats.plot([], [], 'r-', label='Errores (%)')
        
        self.ax_stats.set_xlim(0, 60)  # Últimos 60 segundos
        self.ax_stats.set_ylim(0, 100)
        self.ax_stats.set_xlabel('Tiempo (s)')
        self.ax_stats.set_ylabel('Estadísticas')
        self.ax_stats.set_title('Estadísticas de Comunicación')
        self.ax_stats.grid(True, alpha=0.3)
        self.ax_stats.legend()
        
        # Variables de análisis
        self.analysis = {
            'threshold_voltage': 0,
            'max_current': 0,
            'dynamic_resistance': 0,
            'data_rate': 0,
            'last_analysis_time': time.time()
        }
        
        # Estadísticas temporales
        self.stats_time = []
        self.stats_rate = []
        self.stats_errors = []
        
        plt.tight_layout()
    
    def add_data_point(self, voltage: float, current: float, timestamp: float = None):
        """Agrega punto de datos y actualiza análisis"""
        if timestamp is None:
            timestamp = time.time()
        
        # Agregar datos
        self.voltages.append(voltage)
        self.currents.append(current)
        self.timestamps.append(timestamp)
        self.raw_data.append({'v': voltage, 'i': current, 't': timestamp})
        
        # Limitar número de puntos
        if len(self.voltages) > self.max_points:
            self.voltages.pop(0)
            self.currents.pop(0)
            self.timestamps.pop(0)
        
        # Actualizar análisis cada 10 puntos
        if len(self.voltages) % 10 == 0:
            self._update_analysis()
    
    def _update_analysis(self):
        """Actualiza análisis en tiempo real"""
        if len(self.voltages) < 10:
            return
        
        v_array = np.array(self.voltages)
        i_array = np.array(self.currents)
        
        # Voltaje de umbral (donde I > 10 μA)
        threshold_indices = np.where(i_array > 10)[0]
        if len(threshold_indices) > 0:
            self.analysis['threshold_voltage'] = v_array[threshold_indices[0]]
        
        # Corriente máxima
        self.analysis['max_current'] = np.max(i_array)
        
        # Resistencia dinámica (región lineal)
        if len(v_array) > 20:
            # Tomar últimos 20 puntos para resistencia dinámica
            v_recent = v_array[-20:]
            i_recent = i_array[-20:]
            
            if np.std(i_recent) > 1:  # Solo si hay variación significativa
                # Calcular pendiente dI/dV
                slope = np.gradient(i_recent, v_recent)
                avg_slope = np.mean(slope[slope > 0])
                if avg_slope > 0:
                    self.analysis['dynamic_resistance'] = 1000 / avg_slope  # en ohms
        
        # Calcular tasa de datos
        current_time = time.time()
        time_diff = current_time - self.analysis['last_analysis_time']
        if time_diff > 0:
            self.analysis['data_rate'] = 10 / time_diff  # 10 puntos / tiempo
        
        self.analysis['last_analysis_time'] = current_time
    
    def update_plot(self):
        """Actualiza el gráfico"""
        if len(self.voltages) == 0:
            return
        
        # Actualizar gráfico principal
        self.line_iv.set_data(self.voltages, self.currents)
        
        # Mostrar solo algunos puntos para claridad
        if len(self.voltages) > 10:
            step = max(1, len(self.voltages) // 50)
            v_points = self.voltages[::step]
            i_points = self.currents[::step]
            self.points_iv.set_data(v_points, i_points)
        
        # Ajustar límites automáticamente
        if len(self.voltages) > 5:
            max_v = max(self.voltages)
            max_i = max(self.currents)
            
            self.ax_main.set_xlim(0, max(max_v * 1.1, 1))
            self.ax_main.set_ylim(0, max(max_i * 1.1, 100))
        
        # Actualizar título con estadísticas
        title = f"Curva I-V LED - Puntos: {len(self.voltages)}"
        if self.analysis['threshold_voltage'] > 0:
            title += f" | Vth: {self.analysis['threshold_voltage']:.2f}V"
        if self.analysis['max_current'] > 0:
            title += f" | Imax: {self.analysis['max_current']:.1f}μA"
        
        self.ax_main.set_title(title)
        
        # Actualizar estadísticas
        self._update_stats_plot()
    
    def _update_stats_plot(self):
        """Actualiza gráfico de estadísticas"""
        current_time = time.time()
        
        # Agregar estadísticas actuales
        self.stats_time.append(current_time)
        self.stats_rate.append(self.analysis['data_rate'])
        self.stats_errors.append(0)  # Se puede conectar con ESP32Communication
        
        # Mantener solo últimos 60 segundos
        cutoff_time = current_time - 60
        while self.stats_time and self.stats_time[0] < cutoff_time:
            self.stats_time.pop(0)
            self.stats_rate.pop(0)
            self.stats_errors.pop(0)
        
        if len(self.stats_time) > 1:
            # Convertir a tiempo relativo
            rel_time = [(t - self.stats_time[0]) for t in self.stats_time]
            
            self.line_rate.set_data(rel_time, self.stats_rate)
            self.line_errors.set_data(rel_time, self.stats_errors)
            
            # Ajustar límites
            if rel_time:
                self.ax_stats.set_xlim(0, max(rel_time[-1], 10))
                if self.stats_rate:
                    self.ax_stats.set_ylim(0, max(max(self.stats_rate) * 1.1, 10))
    
    def save_data(self, filename: str = None) -> str:
        """Guarda datos en archivo CSV"""
        if filename is None:
            timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
            filename = f"led_curve_{timestamp}.csv"
        
        try:
            with open(filename, 'w', newline='', encoding='utf-8') as f:
                writer = csv.writer(f)
                writer.writerow(['Timestamp', 'Voltage_V', 'Current_uA'])
                
                for i in range(len(self.voltages)):
                    writer.writerow([
                        self.timestamps[i],
                        self.voltages[i],
                        self.currents[i]
                    ])
            
            print(f"Datos guardados en: {filename}")
            return filename
            
        except Exception as e:
            print(f"Error guardando datos: {e}")
            return ""
    
    def get_analysis_report(self) -> dict:
        """Genera reporte de análisis"""
        if len(self.voltages) == 0:
            return {}
        
        v_array = np.array(self.voltages)
        i_array = np.array(self.currents)
        
        report = {
            'total_points': len(self.voltages),
            'voltage_range': {'min': np.min(v_array), 'max': np.max(v_array)},
            'current_range': {'min': np.min(i_array), 'max': np.max(i_array)},
            'threshold_voltage': self.analysis['threshold_voltage'],
            'max_current': self.analysis['max_current'],
            'dynamic_resistance': self.analysis['dynamic_resistance'],
            'measurement_duration': self.timestamps[-1] - self.timestamps[0] if len(self.timestamps) > 1 else 0
        }
        
        return report
    
    def print_analysis(self):
        """Imprime análisis actual"""
        report = self.get_analysis_report()
        
        print("\n" + "="*50)
        print("ANÁLISIS DE CURVA I-V DEL LED")
        print("="*50)
        
        if report:
            print(f"Puntos medidos: {report['total_points']}")
            print(f"Rango de voltaje: {report['voltage_range']['min']:.3f} - {report['voltage_range']['max']:.3f} V")
            print(f"Rango de corriente: {report['current_range']['min']:.2f} - {report['current_range']['max']:.2f} μA")
            
            if report['threshold_voltage'] > 0:
                print(f"Voltaje de umbral: {report['threshold_voltage']:.3f} V")
            
            print(f"Corriente máxima: {report['max_current']:.2f} μA")
            
            if report['dynamic_resistance'] > 0:
                print(f"Resistencia dinámica: {report['dynamic_resistance']:.1f} Ω")
            
            if report['measurement_duration'] > 0:
                print(f"Duración de medición: {report['measurement_duration']:.1f} s")
        
        print("="*50 + "\n")

print("Clase OptimizedLEDCurvePlotter cargada exitosamente")

In [ ]:
def run_optimized_led_measurement(port: str = None, duration: float = 300):
    """
    Función principal optimizada para medición de curva LED
    
    Args:
        port: Puerto serial (None para auto-detección)
        duration: Duración máxima en segundos
    """
    
    # Inicializar comunicación
    comm = ESP32CommunicationOptimized(port)
    
    if not comm.connect():
        print("Error: No se pudo conectar con ESP32")
        return None, None
    
    # Inicializar visualizador
    plotter = OptimizedLEDCurvePlotter()
    
    # Configurar animación
    def animate(frame):
        # Obtener datos
        data = comm.get_data(timeout=0.01)
        if data:
            plotter.add_data_point(data['vd'], data['id'], data['timestamp'])
        
        # Verificar mensajes del sistema
        sys_msg = comm.get_system_message(timeout=0.01)
        if sys_msg:
            print(f"Sistema: {sys_msg}")
            
            if sys_msg == "COMPLETE":
                print("Medición completada por ESP32")
                plt.close()
                return
        
        # Actualizar gráfico
        plotter.update_plot()
        
        return plotter.line_iv, plotter.points_iv
    
    try:
        print("Esperando inicialización...")
        time.sleep(2)
        
        print("Enviando comando de inicio...")
        comm.send_command('a')
        
        # Crear animación
        ani = animation.FuncAnimation(
            plotter.fig, 
            animate, 
            interval=100,
            blit=False,
            cache_frame_data=False
        )
        
        print("Iniciando medición optimizada...")
        print("Presiona Ctrl+C o cierra la ventana para terminar")
        
        plt.show()
        
    except KeyboardInterrupt:
        print("\nMedición interrumpida por usuario")
        comm.send_command('s')  # Detener medición
    
    except Exception as e:
        print(f"Error durante medición: {e}")
    
    finally:
        # Generar reporte final
        print("\nGenerando reporte final...")
        plotter.print_analysis()
        
        # Mostrar estadísticas de comunicación
        stats = comm.get_stats()
        print(f"Estadísticas de comunicación:")
        print(f"  Paquetes totales: {stats['total_packets']}")
        print(f"  Paquetes válidos: {stats['valid_packets']}")
        print(f"  Errores de checksum: {stats['checksum_errors']}")
        print(f"  Tasa de éxito: {stats['success_rate']:.1f}%")
        
        # Guardar datos
        filename = plotter.save_data()
        
        # Cerrar conexión
        comm.disconnect()
        
        return plotter, comm

def test_esp32_connection(port: str = None) -> bool:
    """
    Prueba la conexión con ESP32 y muestra información del sistema
    """
    comm = ESP32CommunicationOptimized(port)
    
    if not comm.connect():
        return False
    
    try:
        print("Probando comunicación...")
        
        # Enviar comando de información
        comm.send_command('i')
        
        # Esperar respuestas
        start_time = time.time()
        while time.time() - start_time < 5:
            sys_msg = comm.get_system_message(timeout=0.1)
            if sys_msg:
                print(f"Sistema: {sys_msg}")
            
            time.sleep(0.1)
        
        # Probar calibración
        print("\nIniciando calibración...")
        comm.send_command('c')
        
        time.sleep(3)
        
        # Verificar estadísticas
        stats = comm.get_stats()
        print(f"\nEstadísticas de prueba:")
        print(f"  Paquetes procesados: {stats['total_packets']}")
        print(f"  Tasa de éxito: {stats['success_rate']:.1f}%")
        
        return True
        
    except Exception as e:
        print(f"Error en prueba: {e}")
        return False
    
    finally:
        comm.disconnect()

def analyze_saved_data(filename: str):
    """
    Analiza datos guardados desde archivo CSV
    """
    try:
        # Leer datos
        df = pd.read_csv(filename)
        
        if 'Voltage_V' not in df.columns or 'Current_uA' not in df.columns:
            print("Error: Formato de archivo no válido")
            return
        
        voltages = df['Voltage_V'].values
        currents = df['Current_uA'].values
        
        # Crear análisis
        print(f"\nAnalizando datos de: {filename}")
        print(f"Puntos de datos: {len(voltages)}")
        
        # Gráfico mejorado
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(15, 10))
        
        # Curva I-V lineal
        ax1.plot(voltages, currents, 'b-', linewidth=2)
        ax1.set_xlabel('Voltaje (V)')
        ax1.set_ylabel('Corriente (μA)')
        ax1.set_title('Curva I-V - Escala Lineal')
        ax1.grid(True, alpha=0.3)
        
        # Curva I-V logarítmica
        ax2.semilogy(voltages, currents + 0.1, 'r-', linewidth=2)
        ax2.set_xlabel('Voltaje (V)')
        ax2.set_ylabel('Corriente (μA) - Log')
        ax2.set_title('Curva I-V - Escala Logarítmica')
        ax2.grid(True, alpha=0.3)
        
        # Resistencia dinámica
        if len(voltages) > 3:
            resistance = np.gradient(voltages, currents) * 1000000  # en ohms
            resistance = np.clip(resistance, 0, 10000)  # Limitar valores
            
            ax3.plot(voltages[1:], resistance[1:], 'g-', linewidth=2)
            ax3.set_xlabel('Voltaje (V)')
            ax3.set_ylabel('Resistencia Dinámica (Ω)')
            ax3.set_title('Resistencia Dinámica vs Voltaje')
            ax3.grid(True, alpha=0.3)
        
        # Histograma de corrientes
        ax4.hist(currents, bins=30, alpha=0.7, color='orange')
        ax4.set_xlabel('Corriente (μA)')
        ax4.set_ylabel('Frecuencia')
        ax4.set_title('Distribución de Corrientes')
        ax4.grid(True, alpha=0.3)
        
        plt.tight_layout()
        plt.show()
        
        # Análisis estadístico
        print(f"\nAnálisis Estadístico:")
        print(f"  Voltaje: {np.min(voltages):.3f} - {np.max(voltages):.3f} V")
        print(f"  Corriente: {np.min(currents):.2f} - {np.max(currents):.2f} μA")
        print(f"  Corriente promedio: {np.mean(currents):.2f} μA")
        print(f"  Desviación estándar: {np.std(currents):.2f} μA")
        
        # Voltaje de umbral
        threshold_idx = np.where(currents > 10)[0]
        if len(threshold_idx) > 0:
            threshold_v = voltages[threshold_idx[0]]
            print(f"  Voltaje de umbral (10μA): {threshold_v:.3f} V")
        
    except Exception as e:
        print(f"Error analizando datos: {e}")

# Función de conveniencia
def quick_measurement():
    """Función rápida para iniciar medición"""
    print("=== MEDICIÓN RÁPIDA DE CURVA LED ===")
    print("Asegúrate de que el ESP32 esté conectado y el circuito armado")
    input("Presiona Enter para continuar...")
    
    return run_optimized_led_measurement()

print("\n" + "="*60)
print("SISTEMA OPTIMIZADO DE MEDICIÓN LED CARGADO")
print("="*60)
print("Funciones disponibles:")
print("  - quick_measurement(): Medición rápida")
print("  - run_optimized_led_measurement(port): Medición completa")
print("  - test_esp32_connection(port): Probar conexión")
print("  - analyze_saved_data(filename): Analizar datos guardados")
print("="*60)

# Ejemplo de uso automático (descomenta para ejecutar)
# plotter, comm = quick_measurement()

# Resumen del Proyecto: Medición de Curva I-V del Diodo LED

## Comparación entre Versiones

### 📜 Código Original del Profesor
**Características:**
- Código base proporcionado en clase
- Comunicación serial básica
- Errores de sintaxis intencionalmente preservados
- Resistencia de 330Ω
- Sin manejo de errores
- Visualización simple

### 🔧 Código Arduino Optimizado (ESP32_CurvaLED.ino)
**Mejoras implementadas:**
- ✅ Máquina de estados robusta
- ✅ Protocolo de comunicación con checksum
- ✅ Buffer circular para datos
- ✅ Filtrado de ruido avanzado (16 muestras promediadas)
- ✅ Múltiples comandos ('a', 's', 'r', 'i', 'c')
- ✅ Calibración automática de ADC
- ✅ Manejo de timeouts y errores
- ✅ Configuración optimizada (12-bit ADC, 11dB attenuation)

### 🐍 Código Python Optimizado
**Características avanzadas:**
- ✅ Comunicación threaded (no bloquea la interfaz)
- ✅ Auto-detección de puerto ESP32
- ✅ Verificación de integridad de datos (checksum)
- ✅ Visualización en tiempo real con estadísticas
- ✅ Análisis automático de la curva I-V
- ✅ Guardado automático en CSV
- ✅ Manejo robusto de errores y reconexión
- ✅ Múltiples tipos de gráficos (lineal, logarítmico, resistencia dinámica)
- ✅ Cálculo automático de parámetros del LED (Vth, resistencia dinámica)

In [ ]:
# ============================================================================
# EJECUCIÓN AUTOMÁTICA - DESCOMENTA PARA USAR
# ============================================================================

# Opción 1: Medición rápida automática
# plotter, comm = quick_measurement()

# Opción 2: Solo probar conexión
# test_esp32_connection()

# Opción 3: Medición con puerto específico
# plotter, comm = run_optimized_led_measurement(port="/dev/ttyUSB0")

# Opción 4: Analizar datos existentes (cambiar nombre de archivo)
# analyze_saved_data("led_curve_20250530_143022.csv")

print("\n🔬 SISTEMA DE MEDICIÓN LED LISTO")
print("Descomenta una de las líneas de arriba para ejecutar")
print("O ejecuta manualmente las funciones disponibles:")
print("  - quick_measurement()")
print("  - test_esp32_connection()")
print("  - run_optimized_led_measurement(port)")
print("  - analyze_saved_data(filename)")
print("\n📋 RECUERDA:")
print("1. Conectar ESP32 y subir el código Arduino")
print("2. Armar el circuito según el diagrama")
print("3. Verificar que la resistencia sea de 1kΩ")
print("4. Ejecutar una de las funciones de medición")
print("\n🎯 ¡Listo para medir curvas I-V de LEDs!")